In [83]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import os
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.subplots as sp
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import plotly.express as px
import tslearn.clustering as tsc
import statsmodels.api as sm
import patsy
import researchpy as rp
from sklearn import datasets 
from tabulate import tabulate
import importlib
import regression_formulas as rf, clustering_processing as cp
import pickle
base_directory = "C:/Users/CEEM04/OneDrive-Emily/OneDrive - UNSW/edpdatacleaning"
data_directory = os.path.join(base_directory, 'edp_data')
cleaned_data_directory = os.path.join(base_directory, 'resampled_30min_data')

In [84]:
testing_df = pd.read_csv(os.path.join(base_directory, 'combined_aircon_profiles_testing.csv'))

# create new df, drop timestamp column
testing_df_filtered = testing_df.drop(columns=['Timestamp'])
# unique site ids in testing_df
unique_site_ids = testing_df_filtered.columns.unique()

In [85]:
importlib.reload(cp)

survey_df = pd.read_csv(os.path.join(base_directory, 'survey/survey_with_station.csv'))  # Change this to your output directory
survey_df.rename(columns={'edp_site_id': 'site_ID'}, inplace=True)

# filter for unique_site_ids from testing df
survey_df_testing = survey_df[survey_df['site_ID'].isin(unique_site_ids)]
print(survey_df_testing.shape)

(48, 15)


In [ ]:
print(survey_df_testing)

In [86]:
importlib.reload(rf)
weather_folder_path = os.path.join(base_directory, 'weather/bom_data_max_min')  # Change this to your output directory

# Run the merger
weather_survey_df_testing = rf.match_temperatures_to_sites(survey_df_testing, weather_folder_path)

# print rows where site_id is W0266
#print(weather_survey_df_testing[weather_survey_df_testing['site_ID'] == 'W0266'])

c:\Users\CEEM04\OneDrive-Emily\OneDrive - UNSW\edpdatacleaning\regression_formulas.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\CEEM04\OneDrive-Emily\OneDrive - UNSW\edpdatacleaning\regression_formulas.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



printing weather df after reindexed
          date  station_number  max_air_temperature  min_air_temperature
294 2023-10-22           85296                 17.5                  9.4
295 2023-10-23           85296                 19.3                  9.4
296 2023-10-24           85296                 26.8                 10.3
297 2023-10-25           85296                 13.7                  5.2
298 2023-10-26           85296                 14.1                  4.5
299 2023-10-27           85296                 15.4                  5.5
float64
printing df interpolated
          date  station_number  max_air_temperature  min_air_temperature
294 2023-10-22           85296                 17.5                  9.4
295 2023-10-23           85296                 19.3                  9.4
296 2023-10-24           85296                 26.8                 10.3
297 2023-10-25           85296                 13.7                  5.2
298 2023-10-26           85296                 14.1    

In [87]:
with open("multinomial_logit_model.pkl", "rb") as file:
    MN_model = pickle.load(file)

In [88]:

df = weather_survey_df_testing

# W0220 has no aircon type data
# drop any rows with site = W0220
df = df[df['site_ID'] != 'W0220']


#print(df[df['site_ID'] == 'W0266'].tail())
print(df[df.isna().any(axis=1)].head(10))
# Convert categorical variables like 'construction_type' and 'climate_zone' to categorical
df['property_construction'] = df['property_construction'].astype('category')
df['aircon_type_simplified'] = df['aircon_type_simplified'].astype('category')
df['num_bedrooms'] = pd.to_numeric(df['num_bedrooms'], errors='coerce').fillna(5)

# if number of bedrooms is "7 or more", convert to 7
df['num_bedrooms'] = df['num_bedrooms'].replace('7 or more', 7)
df['num_bedrooms'] = df['num_bedrooms'].astype('int')
df['num_occupants'] = df['num_occupants'].replace(['7 or more', '7 or more '], 7)



Empty DataFrame
Columns: [date, weather_station_number, max_air_temperature, min_air_temperature, site_ID, postcode, state, property_construction, property_construction_other, property_star_rating, num_bedrooms, num_occupants, aircon_type, aircon_type_simplified, num_rooms_aircon, num_rooms_heated, climate_zone, weather_station_name]
Index: []


C:\Users\CEEM04\AppData\Local\Temp\ipykernel_8724\2120841179.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\CEEM04\AppData\Local\Temp\ipykernel_8724\2120841179.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\CEEM04\AppData\Local\Temp\ipykernel_8724\2120841179.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [93]:

#print row where num_occupants is NA
df['num_occupants'] = df['num_occupants'].astype('int')
df['climate_zone'] = df['climate_zone'].astype('category')

def get_season(month):
    if month in [12, 1, 2]:
        return 'Summer'
    elif month in [3, 4, 5]:
        return 'Autumn'
    elif month in [6, 7, 8]:
        return 'Winter'
    elif month in [9, 10, 11]:
        return 'Spring'

# Function to determine if the day is a weekend or weekday
def get_weekday_or_weekend(date):
    # Weekdays are Monday (0) to Friday (4), weekends are Saturday (5) and Sunday (6)
    if date.weekday() >= 5:
        return 'WE'  # Weekend
    else:
        return 'WD'  # Weekday

# Convert date to datetime if needed
df['date'] = pd.to_datetime(df['date'])

# Then create the new columns
df['season'] = df['date'].dt.month.apply(get_season)
df['day_type'] = df['date'].apply(get_weekday_or_weekend)


# remove column named 'index'
testing_regression_df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# write df to csv
if not os.path.exists(os.path.join(base_directory, 'df_for_regression_testing.csv')):
    testing_regression_df.to_csv(os.path.join(base_directory, 'df_for_regression_testing.csv'), index=False)
    print(f'File written: df_for_regression_testing.csv')
else:
    print('File already exists!')


C:\Users\CEEM04\AppData\Local\Temp\ipykernel_8724\3205465271.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\CEEM04\AppData\Local\Temp\ipykernel_8724\3205465271.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\CEEM04\AppData\Local\Temp\ipykernel_8724\3205465271.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [96]:
testing_regression_df = pd.read_csv(os.path.join(base_directory, 'df_for_regression_testing.csv'))

In [ ]:
predicted_labels = MN_model.predict(testing_regression_df)
predicted_probabilities = MN_model.predict_proba(testing_regression_df)